# Import the required libraries

In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from apsimNGpy.senstivity.sensitivity import ConfigProblem, run_sensitivity

# Define APSIM model inputs as follows

In [2]:
params = {
       ".Simulations.Simulation.Field.Sow using a variable rule?Population": (2, 10),
       ".Simulations.Simulation.Field.Fertilise at sowing?Amount": (0, 300),
       ".Simulations.Simulation.Field.Maize.CultivarFolder.Dekalb_XL82?[Leaf].Photosynthesis.RUE.FixedValue": (
           1.2, 2.2),
   }

The base parameter path and the specific parameter name or sub-path are separated using one of the following delimiters: ?, ::, or @.

The values associated with each parameter define the lower and upper bounds of the sampling space used during sensitivity analysis.

# Initialize the problem as follows:

In [3]:
runner = ConfigProblem(
    base_model="Maize",
    params=params,
    outputs=["Yield", "Maize.AboveGround.N"],
)

SALib provides an object-oriented interface that simplifies the sensitivity analysis triad of sampling, evaluation, and analysis. In apsimNGpy, this workflow is further streamlined by wrapping all three steps into a single method, run_sensitivity().



In [4]:
Si_sobol = run_sensitivity(
    runner,
    method="sobol",
    N=2 ** 6,  # ← base sample size should be power of 2
    engine = 'csharp',
    sample_options={
        "calc_second_order": False,
        "skip_values": 1024,
         "seed": 42,
    },
    analyze_options={
        "conf_level": 0.95,
        "num_resamples": 1000,
        "print_to_console": True,
        "calc_second_order": False,
    },
)

Processing 320 jobs wait.. ██████████ 100%  >> completed (elapsed=>06:34, eta=>00:00) 
loading data ██████████ 100% (70/70) >> completed (elapsed=>00:02, eta=>00:00) 


                                            ST   ST_conf
Population                            0.224747  0.145908
Amount                                0.600854  0.310677
[Leaf].Photosynthesis.RUE.FixedValue  0.404326  0.168481
                                            S1   S1_conf
Population                            0.113107  0.158142
Amount                                0.454978  0.300406
[Leaf].Photosynthesis.RUE.FixedValue  0.261608  0.222181
                                            ST   ST_conf
Population                            0.195857  0.119274
Amount                                0.712476  0.298475
[Leaf].Photosynthesis.RUE.FixedValue  0.281958  0.132216
                                            S1   S1_conf
Population                            0.105329  0.153433
Amount                                0.575180  0.324670
[Leaf].Photosynthesis.RUE.FixedValue  0.176232  0.180163


The returned object is an instance of the SALib.util.problem.ProblemSpec class. When evaluated in the Python console, this object displays a summary of the problem definition and the computed sensitivity indices, as shown below.

# Converting outputs to pandas

In [5]:
Si_sobol.to_df()

[[                                            ST   ST_conf
  Population                            0.224747  0.145908
  Amount                                0.600854  0.310677
  [Leaf].Photosynthesis.RUE.FixedValue  0.404326  0.168481,
                                              S1   S1_conf
  Population                            0.113107  0.158142
  Amount                                0.454978  0.300406
  [Leaf].Photosynthesis.RUE.FixedValue  0.261608  0.222181],
 [                                            ST   ST_conf
  Population                            0.195857  0.119274
  Amount                                0.712476  0.298475
  [Leaf].Photosynthesis.RUE.FixedValue  0.281958  0.132216,
                                              S1   S1_conf
  Population                            0.105329  0.153433
  Amount                                0.575180  0.324670
  [Leaf].Photosynthesis.RUE.FixedValue  0.176232  0.180163]]

Key important attributes for samples, model results and analyses extraction available on the returned instance are shown below.

In [13]:
print( Si_sobol.samples[:10])
print( Si_sobol.results[:10])# same as APSIM outputs
print( Si_sobol.analysis)

[[  7.79680204 217.38007663   1.92460026]
 [  3.8583405  208.1493074    1.80152333]
 [  4.08012103 198.91853817   1.67844641]
 [  8.01858257 189.68776894   1.55536949]
 [  8.04295589 180.45699971   1.43229256]
 [  4.10449435 171.22623047   1.30921564]
 [  3.83396719 161.99546124   1.21386128]
 [  7.77242873 152.76469201   1.33693821]
 [  8.28910974 143.53392278   1.46001513]
 [  4.3506482  134.30315355   1.58309205]]
[[56657.05443862   128.96879572]
 [44193.00499912    95.0857142 ]
 [42333.34875365    90.94491259]
 [45117.10956259   102.81235105]
 [40880.42830486    93.13595534]
 [32184.60788019    68.18285935]
 [28557.16287038    60.3197544 ]
 [37062.36821457    84.0171209 ]
 [41955.05257056    95.95766543]
 [40792.41704208    87.52846484]]
{'Y1': {'S1': [np.float64(0.1403930384140257), np.float64(0.3073745950747107), np.float64(0.3966462630790386)], 'ST': [np.float64(0.2611888823350895), np.float64(0.5632569490136345), np.float64(0.6112463294153863)], 'S1_conf': [np.float64(0.2060771

Attribute ``apsim_results`` allows you to extract the evaluation or APSIM simulated results

In [7]:
Si_sobol.apsim_results

,Yield,Maize.AboveGround.N,source_table,ID,Population,Amount,[Leaf].Photosynthesis.RUE.FixedValue,MetaProcessID
143,46625.065683,102.826004,Report,0,5.438373,201.254380,1.724356,35020
86,44521.143800,96.383571,Report,1,4.403738,201.254380,1.724356,57436
73,46445.654480,102.489966,Report,2,5.438373,279.726169,1.724356,2412
6,33198.196011,71.407587,Report,3,5.438373,201.254380,1.268925,4236
282,31571.305504,67.024924,Report,4,4.403738,279.726169,1.268925,16304
...,...,...,...,...,...,...,...,...
33,21317.500208,34.488829,Report,315,5.289632,9.417092,2.093965,37260
71,21243.206928,35.834347,Report,316,2.201366,9.417092,2.093965,31996
140,26002.091121,42.693403,Report,317,5.289632,20.950835,2.093965,50120
22,21462.785564,34.385157,Report,318,5.289632,9.417092,1.408605,18820


We can try another method known as Morris. The Morris method is typically used as a screening tool to identify influential parameters with relatively low computational cost. It is well suited for high-dimensional problems where the goal is to rank parameters rather than quantify precise sensitivities.

In [8]:
Si_morris = run_sensitivity(
      runner,
      method="morris", n_cores=10,
      sample_options={
          'seed': 42,
          "num_levels": 6,
          "optimal_trajectories": 6,
      },
      analyze_options={
          'conf_level': 0.95,
          "num_resamples": 1000,
          "print_to_console": True,
          'seed': 42
      },
  )

APSIM running ██████████ 100% (24/24) >> completed (elapsed=>00:48, eta=>00:00) 


                                                mu       mu_star  \
Population                            14468.626235  14954.893827   
Amount                                11747.876921  11850.723789   
[Leaf].Photosynthesis.RUE.FixedValue  19869.082620  20022.165155   

                                             sigma  mu_star_conf  
Population                            18563.179845  13230.193075  
Amount                                14119.888108  10382.186160  
[Leaf].Photosynthesis.RUE.FixedValue  14720.234695  10689.468190  
                                             mu    mu_star      sigma  \
Population                            34.717247  34.717247  39.508733   
Amount                                40.866092  40.895361  37.585802   
[Leaf].Photosynthesis.RUE.FixedValue  45.606532  47.466331  33.527863   

                                      mu_star_conf  
Population                               29.027859  
Amount                                   27.735131  
[Leaf].

The final sensitivity analysis method demonstrated in this tutorial is the FAST method. FAST is a variance-based approach that estimates the influence of each input parameter by systematically varying inputs across the parameter space and analyzing the resulting model response.

In [9]:
si_fast = run_sensitivity(
    runner,
    method="fast",
    sample_options={
        "M": 2,

    },
    analyze_options={
        'conf_level': 0.95,
        "num_resamples": 1000,
        "print_to_console": True,
    },
)

APSIM running ██████████ 100% (195/195) >> completed (elapsed=>04:08, eta=>00:00) 
loading data ██████████ 100% (18/18) >> completed (elapsed=>00:02, eta=>00:00) 


database deleted
                                            S1        ST   S1_conf   ST_conf
Population                            0.140393  0.261189  0.206077  0.154489
Amount                                0.307375  0.563257  0.206815  0.156095
[Leaf].Photosynthesis.RUE.FixedValue  0.396646  0.611246  0.209430  0.159443
                                            S1        ST   S1_conf   ST_conf
Population                            0.124625  0.237254  0.212674  0.155253
Amount                                0.438772  0.661299  0.203179  0.158683
[Leaf].Photosynthesis.RUE.FixedValue  0.333819  0.527898  0.205883  0.157800


Ideally, the analysis could begin with the Morris method to screen out less influential factors, particularly when a large number of inputs are under consideration. However, even without this initial screening, the results are consistent across methods.

In this case, both FAST and Sobol analyses indicate that yield is more sensitive to nitrogen fertilizer rate than to population density.

To extend the analysis beyond the Sobol, Morris, and FAST methods, follow the same workflow demonstrated in the first example and consult the SALib documentation for additional sensitivity analysis techniques and their usage.

- Sensitivity module was Added in version 1.0.0.
- version 1.1.0 provided improvements to the API by allowing specification of back-end simulation

# References

Iwanaga, T., Usher, W., & Herman, J. (2022). Toward SALib 2.0: Advancing the accessibility and interpretability of global sensitivity analyses. Socio-Environmental Systems Modelling, 4, 18155. doi:10.18174/sesmo.18155

Herman, J. and Usher, W. (2017) SALib: An open-source Python library for sensitivity analysis. Journal of Open Source Software, 2(9). doi:10.21105/joss.00097

